In [82]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# Any results you write to the current directory are saved as output.

/kaggle/input/items.csv


In [83]:
dataset = pd.read_csv("../input/items.csv", delimiter=",", header=None, usecols=[1], skiprows=[0])
print(dataset)

         1
0    A;C;D
1    B;C;E
2  A;B;C;E
3      B;E


In [84]:
buffer = []
for i in range(0,4):
    buffer.append([str(dataset.values[i,j]) for j in range(1)])

datas = []
for i in buffer:
    for j in i:
        datas.append(str(j).split(";"))
print(datas)

[['A', 'C', 'D'], ['B', 'C', 'E'], ['A', 'B', 'C', 'E'], ['B', 'E']]


In [85]:
def createC1(dataSet):
    C1 = []
    for transaction in dataSet:
        for item in transaction:
            if not [item] in C1:
                C1.append([item])
                
    C1.sort()
    return list(map(frozenset, C1))

In [86]:
def scanD(D, Ck, minSupport):
    ssCnt = {}
    for tid in D:
        for can in Ck:
            if can.issubset(tid):
                if not can in ssCnt: ssCnt[can]=1
                else: ssCnt[can] += 1
    numItems = float(len(D))
    retList = []
    supportData = {}
    for key in ssCnt:
        support = ssCnt[key]/numItems
        if support >= minSupport:
            retList.insert(0,key)
        supportData[key] = support
    return retList, supportData

In [87]:
C1 = createC1(datas)
C1

[frozenset({'A'}),
 frozenset({'B'}),
 frozenset({'C'}),
 frozenset({'D'}),
 frozenset({'E'})]

In [88]:
D = list(map(set,datas))
D

[{'A', 'C', 'D'}, {'B', 'C', 'E'}, {'A', 'B', 'C', 'E'}, {'B', 'E'}]

In [89]:
L1,suppDat0 = scanD(D,C1,0.5)
L1

[frozenset({'E'}), frozenset({'B'}), frozenset({'C'}), frozenset({'A'})]

In [90]:
def aprioriGen(Lk, k): 
    retList = []
    lenLk = len(Lk)
    for i in range(lenLk):
        for j in range(i+1, lenLk): 
            L1 = list(Lk[i])[:k-2]; L2 = list(Lk[j])[:k-2]
            L1.sort(); L2.sort()
            if L1==L2: 
                retList.append(Lk[i] | Lk[j])
    return retList

In [91]:
def apriori(dataSet, minSupport = 0.5):
    C1 = createC1(dataSet)
    D = list(map(set, dataSet))
    L1, supportData = scanD(D, C1, minSupport)
    L = [L1]
    k = 2
    while (len(L[k-2]) > 0):
        Ck = aprioriGen(L[k-2], k)
        Lk, supK = scanD(D, Ck, minSupport)
        supportData.update(supK)
        L.append(Lk)
        k += 1
    return L, supportData

In [92]:
L,suppData = apriori(datas)
L

[[frozenset({'E'}), frozenset({'B'}), frozenset({'C'}), frozenset({'A'})],
 [frozenset({'B', 'C'}),
  frozenset({'C', 'E'}),
  frozenset({'B', 'E'}),
  frozenset({'A', 'C'})],
 [frozenset({'B', 'C', 'E'})],
 []]

In [93]:
aprioriGen(L[0],2)

[frozenset({'B', 'E'}),
 frozenset({'C', 'E'}),
 frozenset({'A', 'E'}),
 frozenset({'B', 'C'}),
 frozenset({'A', 'B'}),
 frozenset({'A', 'C'})]

In [94]:
L,suppData= apriori(datas,minSupport=0.5)
suppData

{frozenset({'A'}): 0.5,
 frozenset({'C'}): 0.75,
 frozenset({'D'}): 0.25,
 frozenset({'B'}): 0.75,
 frozenset({'E'}): 0.75,
 frozenset({'A', 'C'}): 0.5,
 frozenset({'B', 'E'}): 0.75,
 frozenset({'C', 'E'}): 0.5,
 frozenset({'B', 'C'}): 0.5,
 frozenset({'A', 'E'}): 0.25,
 frozenset({'A', 'B'}): 0.25,
 frozenset({'B', 'C', 'E'}): 0.5}